# Build and Test Experiments

## `x86` assembly with `nostd` and `kernel32.lib`

Uses my fantastic one of a kind build script `BuildO`!

Because I will never be persuaded to use Visual Studio again! And besides, you
will never replicate my build process on your own machine, so why should you be
able to understand my build script? Creating unintelligible build scripts seems
to be lesson one in assembly. Yay for assembly.

In [1]:
import os
import struct
from pathlib import Path

from buildo import BuildO, LibO, MSVC_Linker, NASM_Assembler, cleanup

In [2]:
cleanup(".")

The following files will be deleted:
    01_exit42\main.obj
    02_exit_arg\main.obj
    03_include_test\main.obj
    04_hello_world\main.obj
    05_hello_world_repeat\main.obj
    07_write_out_float\main.obj
    01_exit42\main.exe
    02_exit_arg\main.exe
    03_include_test\main.exe
    04_hello_world\main.exe
    05_hello_world_repeat\main.exe
    07_write_out_float\main.exe
Done


In [3]:
buildo = BuildO(
    linko = MSVC_Linker(
        r"C:\Program Files\Microsoft Visual Studio\2022\Community\VC\Tools\MSVC\14.40.33807\bin\Hostx86\x86\link.exe"
    )
    .with_no_default_lib()
    .with_entry_point("start")
    .with_libo(
        LibO(r"C:\Program Files (x86)\Windows Kits\10\Lib\10.0.26100.0\um\x86")
        .with_lib("Kernel32.Lib")
    ),
    assemblo = NASM_Assembler(
        os.path.expandvars(r"%LOCALAPPDATA%\bin\NASM\nasm.EXE")
    ),
)

In [4]:
buildo.build("./01_exit42/main.asm").run_print()

Return Code: 42


In [5]:
# this demo gets a pointer to the arguments address, and simply loads the first 4 bytes into
# the return code as an integer.
# the result is the start of the name of the exe file.
# Or it may be the start of the full path depending on how it was called.
result = buildo.build("./02_exit_arg/main.asm").run()
# unpack 4 bytes as an int each from 32 bit integer string
chars =  int(result.returncode).to_bytes(4, byteorder='little', signed=False)
print(chars)

b'"M:\\'


In [6]:
buildo.build("./03_include_test/main.asm").run_print()

     StdOut: Hello, World!
 


In [7]:
buildo.build("./04_hello_world/main.asm").run_print()

     StdOut: Hello, World!
 


In [8]:
buildo.build("./05_hello_world_repeat/main.asm").run_print()

     StdOut: Hello, World!
 Hello, World!
 Hello, World!
 Hello, World!
 Hello, World!
 Hello, World!
 Hello, World!
 Hello, World!
 Hello, World!
 Hello, World!
 


In [9]:
# Fail; seems impossible to get prinf to import ??

# buildo.with_linko(
#     buildo
#     .linko
#     # .with_libo(
#     #     LibO(r"C:\Program Files\Microsoft Visual Studio\2022\Community\VC\Tools\MSVC\14.40.33807\lib\x86")
#     #     .with_lib("libvcruntime.lib") # printf ????!
#     # )
#     .with_default_lib()
# ).with_verbose(True).build("06_c_msvcrt_print_float_test/main.asm").run_print()

In [10]:
buildo.build("./07_write_out_float/main.asm").run_print()
print(Path("./07_write_out_float/cheese_out.txt").read_text())
print("float output: ",struct.unpack(
    "d",
    Path("./07_write_out_float/cheese_out.txt").read_bytes()[-8:]
)[0])

"M:\GIT\x86 exploration\07_write_out_float\main.exe"
Hello, World!
Tã¥›Ä 	@
float output:  3.141
